In [7]:
import numpy as np
import cv2
import os
from tensorflow.keras import applications
from tensorflow.contrib.keras import backend as K
from tensorflow.keras.layers import Input, Dense, Flatten, Dropout, Concatenate, BatchNormalization
from tensorflow.keras.models import Model
import tensorflow.keras
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D

from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import load_img,img_to_array
from tensorflow.keras.optimizers import Adam,SGD
from numpy import genfromtxt
from tensorflow.keras.models import load_model
%matplotlib inline
import matplotlib.pyplot as plt
from tensorflow.keras import regularizers
from PIL import Image
import sys
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
import tensorflow as tf
tf.test.gpu_device_name()

''

In [0]:
def custom_age_loss(y_true, y_pred): 
  
    true = K.sum(y_true * K.arange(0, 8, dtype="float32"), axis=-1)
    mean_pred = K.sum(y_pred * K.arange(0, 8, dtype="float32"), axis=-1)
    mean_loss = K.abs(true - mean_pred)    

    total_loss = mean_loss 

    return total_loss

In [0]:
K.clear_session()

input_im = Input(shape=(256,256,3))

# Block 1
x = Conv2D(64, (3, 3),activation='relu',padding='same',name='block1_conv1')(input_im)
x = Conv2D(64, (3, 3),activation='relu',padding='same',name='block1_conv2')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

# Block 2
x = Conv2D(128, (3, 3),activation='relu',padding='same',name='block2_conv1')(x)
x = Conv2D(128, (3, 3),activation='relu',padding='same',name='block2_conv2')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

# Block 3
x = Conv2D(256, (3, 3),activation='relu',padding='same',name='block3_conv1')(x)
x = Conv2D(256, (3, 3),activation='relu',padding='same',name='block3_conv2')(x)
x = Conv2D(256, (3, 3),activation='relu',padding='same',name='block3_conv3')(x)
x = Conv2D(256, (3, 3),activation='relu',padding='same',name='block3_conv4')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

# Block 4
x = Conv2D(512, (3, 3),activation='relu',padding='same',name='block4_conv1')(x)
x = Conv2D(512, (3, 3),activation='relu',padding='same',name='block4_conv2')(x)
x = Conv2D(512, (3, 3),activation='relu',padding='same',name='block4_conv3')(x)
x = Conv2D(512, (3, 3),activation='relu',padding='same',name='block4_conv4')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

# Block 5
x = Conv2D(512, (3, 3),activation='relu',padding='same',name='block5_conv1')(x)
x = Conv2D(512, (3, 3),activation='relu',padding='same',name='block5_conv2')(x)
x = Conv2D(512, (3, 3),activation='relu',padding='same',name='block5_conv3')(x)
x = Conv2D(512, (3, 3),activation='relu',padding='same',name='block5_conv4')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)

# My Mod
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation="relu")(x)
gen = Dense(2, activation="softmax",name='gen')(x)
age = Dense(8, activation="softmax",name='age')(x)

model = Model(inputs = input_im, outputs = [gen,age])


#model.compile(loss='categorical_crossentropy',optimizer='Adam', metrics=['accuracy'])
SVG(model_to_dot(model,show_shapes=True).create(prog='dot', format='svg'))

In [0]:

model.compile(
    optimizer=tf.train.GradientDescentOptimizer (learning_rate=1e-2, ),
    loss = {'gen':'binary_crossentropy','age':custom_age_loss} ,
    metrics=['accuracy']    
)

model = tf.contrib.tpu.keras_to_tpu_model(
    model,
    strategy=tf.contrib.tpu.TPUDistributionStrategy(
        tf.contrib.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
    ))

#model.summary()

In [0]:
# usage : custom_gen(range(0,3000+1))
def custom_gen(_anns,use_range,batch_size):
    while True:
        image_data = []
        label_data_gender = []
        label_data_age = []
        start_indx = np.random.randint(low=use_range[0],high=use_range[-batch_size])
        
        # Read anns
        batch_row = _anns[start_indx:start_indx+batch_size]       
                    
        
        # Read images
        for row in batch_row:
            im = load_img('dataset/'+row[0]+'/coarse_tilt_aligned_face.'+row[2]+'.'+row[1],target_size=(256, 256,3))  #PIL Image
            if np.random.randint(low=0,high=2):
                im = im.transpose(Image.FLIP_LEFT_RIGHT)  
            im = img_to_array(im) # Numpy
            im = im/255.
            image_data.append(im)               
            
            label_data_gender.append(np.fromstring(row[5], sep=' ')) # target gender
            label_data_age.append(np.fromstring(row[6], sep=' ')) # target gender
            
   
        image_data = np.array(image_data)
 
        
        yield( image_data, [label_data_age ])       
        

In [0]:
# Read CSV annotations
train_anns = genfromtxt('anns/'+'train_anns.csv', delimiter=',',dtype=str)
val_anns = genfromtxt('anns/'+'val_anns.csv', delimiter=',',dtype=str)
np.random.shuffle(train_anns)
np.random.shuffle(val_anns)

train_range = range(0,10000+1) # all = 10000
val_range = range(0,1332+1) # all = 1332

print('train_range : ',train_range[0],'-',train_range[-1],\
      '\nval_range',val_range[0],'-',val_range[-1])
batch_size = 128

train_range :  0 - 10000 
val_range 0 - 1332


# TRAIN

In [0]:
history = model.fit_generator(
        custom_gen(train_anns,train_range,batch_size), 
        steps_per_epoch=max(1, len(train_range)//batch_size),
        epochs=30,        
        validation_data=custom_gen(val_anns,val_range,batch_size),
        validation_steps=max(1, len(val_range)//batch_size))


# TEST

In [0]:
gender_dict = ['m','f']
age_dict = ['(0-2)','(4-6)','(8-12)','(15-20)','(25-32)','(38-43)','(48-53)','(60-100)']

true_gen = 0
true_age = 0 
for b in range(0,1):
    image_data = []
    sys.stdout.write("\r" + 'test batch : '+str(b))
    sys.stdout.flush()
    for i in range(8):
        
        #rand_indx = np.random.randint(low=0,high=1333)
        #rowp = val_anns[rand_indx]
        rowp = val_anns[b+i]
        im_ori = load_img('dataset/'+rowp[0]+'/coarse_tilt_aligned_face.'+rowp[2]+'.'+rowp[1],target_size=(256, 256,3))        
        
        if np.random.randint(low=0,high=2):
            im_ori = im_ori.transpose(Image.FLIP_LEFT_RIGHT)            
        im = img_to_array(im_ori)
        
  
        im = im/255. 
        #im = np.expand_dims(im, axis=0)
        image_data.append(im)
    image_data = np.asarray(image_data[:8])
  
    print('shape = ',image_data.shape)   

    [genderp,agep] = model.predict(image_data,use_multiprocessing=False,batch_size=8)
    
   
    
    

    if rowp[4]==gender_dict[genderp.argmax()]:
        true_gen += 1
    if rowp[3]==age_dict[agep.argmax()]:
        true_age += 1


      #print(rowp[4]==gender_dict[genderp.argmax()],'\t',rowp[4],'\t\t|',gender_dict[genderp.argmax()],np.amax(genderp))
      #print(rowp[3]==age_dict[agep.argmax()],'\t',rowp[3],'\t|',age_dict[agep.argmax()],np.amax(agep))

      #display(im_ori)

print('\ncorrect-gender : ','{:d} prob-{:.2f}'.format(true_gen,true_gen/i),\
      '\ncorrect-age : ','{:d} prob-{:.2f}'.format(true_age,true_age/i)) 

In [0]:
def reset_weights(model):
    session = K.get_session()
    for layer in model.layers: 
        if hasattr(layer, 'kernel_initializer'):
            layer.kernel.initializer.run(session=session)

In [0]:
reset_weights(model)

In [0]:
print (model.get_weights())

In [0]:
model.save('my_model_findingage.h5')

INFO:tensorflow:Copying TPU weights to the CPU


In [0]:

plt.plot(history.history['gen_loss'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')

plt.show()

In [0]:
np.savetxt("age_loss.txt", np.array(history.history['age_loss']), delimiter=",")

In [0]:
from google.colab import drive
drive.mount('/content/drive/')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
!cp my_model.h5 '/content/drive/My Drive/Classes/year4/image_process/gender_age/gender_age_cnn/my_model255.h5' 

In [0]:
!unzip '/content/drive/My Drive/Classes/year4/image_process/gender_age/Desktop.zip'
!cp '/content/drive/My Drive/Classes/year4/image_process/gender_age/gender_age_cnn/anns' . -r

In [0]:
!pip install pydot
!apt-get install graphviz


1

In [0]:
y_true = K.variable([0,0,0,0,0,1,0,0])
y_pred = K.variable([0.0,0.8,0.0,0.0,0.0,0.1,0.0,0.1])

K.eval(custom_age_loss(y_true,y_pred))


13.2

2.0

In [0]:
!cp my_model256.h5 '/content/drive/My Drive/Classes/' 

In [0]:
! ls


anns  dataset  drive  my_model256.h5  sample_data


In [0]:
model = load_model('my_model_findingage.h5')

In [0]:
model.summary()
